In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")
import json

In [2]:
import random
from spacy.util import minibatch, compounding
from spacy.training import Example

In [88]:
from pathlib import Path

In [68]:
ff = open("traindata.json",'r',encoding='utf-8')
ss = []
for line in ff:
    ss.append(line)

In [69]:
def filter_anotation(anLst):
    ss_n = []
    prev_ij = []
    
    for i,j,s in anLst:
        if prev_ij == []:
            prev_ij.append([i,j])
            ss_n.append((i,j,s))
        else:
            not_pre = True
            for ii in prev_ij:
                a = ii[0]
                b = ii[1]
                if ((i >= a and i <b) or (j >= a and j <b)) or ((a >= i and a <j) or (b >= i and b <j)):
                    not_pre = False
                    
            if [i,j] not in prev_ij and not_pre:
                prev_ij.append([i,j])
                ss_n.append((i,j,s))
    return ss_n


In [117]:
lr = []
labels = []
for st_l in ss:
    d = json.loads(st_l)
    cont = " ".join(d['content'].split("\n"))
#     cont = " ".join([t.text.lower() for t in nlp.tokenizer(cont) if t.is_alpha])
    cont  = " ".join([sd.text for sd in nlp.tokenizer(cont) if not sd.is_punct])
    
    ets = []
#     print(d['annotation'])
    for ano in d['annotation']:
        if len(ano['label']) != 0:
#             print(ano['points'])
            lbl = "_".join(ano['label'][0].lower().split(" "))
            txt =  " ".join(ano['points'][0]['text'].split('\n'))
            txt = [sd.text for sd in nlp.tokenizer(" ".join(txt.split(","))) if not sd.is_punct]
#             txt = [t.text.lower() for t in nlp.tokenizer(ano['points'][0]['text']) if t.is_alpha]

            for ww in txt:
                s = cont.find(ww)
                e = s + len(ww)
                tpl = (s,e,lbl)
                ets.append(tpl)
#             print(ets," ",ano['label'])
#     ets = filter_anotation(ets)
    ets = list(set(ets))
    if len(ets) != 0:
        lr.append((cont,{'entities':ets}))
        

In [118]:
test_nlp = spacy.blank('en')
ner = test_nlp.create_pipe('ner')
test_nlp.add_pipe('ner')

In [119]:
for _,et in lr:
    for sss in et['entities']:
        lbl = sss[-1]
        ner.add_label(lbl)

In [120]:
test_nlp.pipe_labels

{'ner': []}

In [121]:
optimizer = test_nlp.begin_training()

In [123]:
for _ in range(1):
    losses = {}
    random.shuffle(lr)
    p= 0 
    for raw_text, entity_offsets in lr[2:]:
        doc = test_nlp.make_doc(raw_text)
        print(p)
        p = p +1
        example = Example.from_dict(doc, entity_offsets)
        test_nlp.update([example], sgd=optimizer, drop=0.2,
                   losses=losses)
    print('Losses', losses)

0


ValueError: [E103] Trying to set conflicting doc.ents: '(60, 66, 'companies_worked_at')' and '(60, 66, 'skills')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.

In [132]:
lr[2:3][0]

("Shrishti Chauhan Have total work experience of 2.5 years on Oracle Fusion Middleware SOA, WebLogic and MFT Module.   Bilaspur, Chhattisgarh Email me on Indeed: indeed.com/r/Shrishti- Chauhan/89d7feb4b3957524   Seeking to hone and enhance my technical skills in Oracle Fusion Middleware while working as a professional in challenging and goal oriented environment.   Willing to relocate to: Bengaluru, Karnataka   WORK EXPERIENCE   Technical Consultant   Oracle   Bengaluru, Karnataka   October 2015 to Present   Have total work experience of 2.5 years on Oracle Fusion Middleware SOA, WebLogic and MFT Module.   Have extensively worked on Support, Testing, Cloning, Monitoring and Maintenance support and Enhancement for the E-Commerce Project with multi system module.   Have good understanding on End-to-End Business Process.   Experience in developing and deploying BPEL Processes using technology adapters (DB Adapter, File Adapter, FTP Adapter and JMS Adapter).   Part of a team for developing

In [96]:

output_dir=Path('content/')

# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
test_nlp.meta['name'] = 'my_ner_more'  # rename model
test_nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to content


In [23]:
spacy.training.offsets_to_biluo_tags(nlp.make_doc(lr[3][0]), lr[3][1]['entities'])

C:\Users\pinakin\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Ijas Nizamuddin Associate Consultant - State Stree..." with entities "[(4652, 4716, 'skills'), (4717, 4755, 'skills'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


['B-name',
 'L-name',
 'B-designation',
 'L-designation',
 'O',
 '-',
 '-',
 'O',
 'B-location',
 'L-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '-',
 '-',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-companies_worked_at',
 'L-companies_worked_at',
 'O',
 'O',
 'O',
 '-',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '

In [112]:
dd = test_nlp(lr[20][0])

In [111]:
lr[20][1]

{'entities': [(4511, 4516, 'skills'),
  (4517, 4519, 'skills'),
  (4520, 4526, 'skills'),
  (7, 8, 'skills'),
  (4528, 4532, 'skills'),
  (4544, 4548, 'skills'),
  (4560, 4563, 'skills'),
  (4575, 4579, 'skills'),
  (4591, 4596, 'skills'),
  (4608, 4617, 'skills'),
  (4629, 4634, 'skills'),
  (4646, 4651, 'skills'),
  (4663, 4666, 'skills'),
  (413, 419, 'skills'),
  (4685, 4687, 'skills'),
  (4484, 4489, 'college_name'),
  (4446, 4453, 'college_name'),
  (4456, 4461, 'degree'),
  (68, 70, 'degree'),
  (4403, 4411, 'degree'),
  (4412, 4419, 'degree'),
  (4442, 4445, 'college_name'),
  (4386, 4392, 'degree'),
  (4393, 4399, 'degree'),
  (2011, 2016, 'companies_worked_at'),
  (1573, 1578, 'companies_worked_at'),
  (1546, 1552, 'designation'),
  (2694, 2702, 'designation'),
  (1561, 1569, 'designation'),
  (1553, 1560, 'designation'),
  (100, 101, 'years_of_experience'),
  (127, 131, 'years_of_experience'),
  (60, 67, 'email_address'),
  (17, 25, 'location')]}

In [83]:
# ner.move_names

In [84]:
# for ent in dd.ents:
#     print(ent.text,ent.label_)

In [ ]:
cont = " ".join(d['content'].split("\n"))
cont  = " ".join([sd.text for sd in nlp.tokenizer(cont) if not sd.is_punct])

In [113]:
spacy.displacy.render(dd,style='ent')

In [114]:
dd.ents

(Banglore,
 Indeed:,
 indeed.com/r/Madhuri-,
 Sripathi/04a52a262175111c,
 python,
 Senior,
 project,
 engineer,
 Wipro,
 Cisco,
 software,
 Master,
 degree,
 computer,
 science,
 PES,
 college,
 S.S.C,
 Rajah,
 LINUX,
 (4,
 years),
 UNIX,
 ospf,
 bgp,
 mpls,
 ipsec,
 multicast,
 l2vpn,
 l3vpn,
 tcl,
 (2,
 ADDITIONAL)